                                                 Иззатов Эльшан Али-заде

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Задание 4.5
5. Напишите программу, которая для заданного каталога создает подкаталоги, соответствующие дате создания каждого отдельного файла, и перемещает каждый файл в соответствующий дате каталог.

In [13]:
import os
import shutil
from datetime import datetime
import pytz

def ask_user_action(file_path):
    """
    Запрашивает действие у пользователя при конфликте имен файлов
    """
    while True:
        print(f"\nФайл '{os.path.basename(file_path)}' уже существует.")
        print("1. Перезаписать")
        print("2. Пропустить")
        print("3. Переименовать")
        choice = input("Выберите действие (1-3): ").strip()

        if choice == '1':
            return 'overwrite'
        elif choice == '2':
            return 'skip'
        elif choice == '3':
            return 'rename'
        else:
            print("Некорректный ввод. Попробуйте снова.")

def organize_files_by_time(source_dir):
    """
    Организует файлы в подкаталоги по времени создания с проверкой конфликтов
    """
    try:
        if not os.path.exists(source_dir):
            raise FileNotFoundError(f"Каталог не найден: {source_dir}")

        items = os.listdir(source_dir)
        if not items:
            print("Каталог пуст, нечего сортировать.")
            return

        processed_files = 0
        skipped_files = 0
        renamed_files = 0

        timezone = pytz.timezone('Europe/Moscow')

        for item in items:
            item_path = os.path.join(source_dir, item)

            if not os.path.isfile(item_path):
                continue

            try:
                stat = os.stat(item_path)
                creation_timestamp = stat.st_mtime
                time_obj = datetime.fromtimestamp(creation_timestamp, timezone)
                time_folder = time_obj.strftime("%H_%M")
                actual_time = time_obj.strftime("%H:%M")

                if not (0 <= time_obj.hour < 24 and 0 <= time_obj.minute < 60):
                    raise ValueError(f"Некорректное время файла: {actual_time}")

                target_dir = os.path.join(source_dir, time_folder)
                os.makedirs(target_dir, exist_ok=True)

                new_path = os.path.join(target_dir, item)

                # Обработка случая, когда файл уже существует
                if os.path.exists(new_path):
                    action = ask_user_action(new_path)

                    if action == 'skip':
                        print(f"Файл '{item}' пропущен.")
                        skipped_files += 1
                        continue
                    elif action == 'rename':
                        base, ext = os.path.splitext(item)
                        counter = 1
                        while os.path.exists(new_path):
                            new_name = f"{base}_{counter}{ext}"
                            new_path = os.path.join(target_dir, new_name)
                            counter += 1
                        renamed_files += 1
                        print(f"Файл переименован в '{os.path.basename(new_path)}'")

                shutil.move(item_path, new_path)
                print(f"Файл '{os.path.basename(new_path)}' перемещён в '{time_folder}/' (время создания: {actual_time})")
                processed_files += 1

            except PermissionError:
                print(f"Ошибка доступа: {item} (пропущен)")
                skipped_files += 1
            except Exception as e:
                print(f"Ошибка обработки '{item}': {str(e)}")
                skipped_files += 1

        print(f"\nСтатистика:")
        print(f"Обработано файлов: {processed_files}")
        print(f"Переименовано файлов: {renamed_files}")
        print(f"Пропущено файлов: {skipped_files}")

    except FileNotFoundError as fnf_error:
        print(fnf_error)
    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")

# Установка зависимостей (если нужно)
try:
    import pytz
except ImportError:
    !pip install pytz
    import pytz

# Укажите путь для сортировки
folder_to_organize = '/content/drive/MyDrive/Программирование на языке Python/Lab_Python_4_PMI/5_задание'

print("\nНачинаем сортировку файлов по времени создания...")
organize_files_by_time(folder_to_organize)
print("\nПроцесс завершен! Проверьте результат.")


Начинаем сортировку файлов по времени создания...

Файл 'math_students.xlsx' уже существует.
1. Перезаписать
2. Пропустить
3. Переименовать
Выберите действие (1-3): 2
Файл 'math_students.xlsx' пропущен.

Статистика:
Обработано файлов: 0
Переименовано файлов: 0
Пропущено файлов: 1

Процесс завершен! Проверьте результат.


# Задание 5.2
2. Найти максимальный элемент в одномерном массиве x среди элементов, перед которыми стоит нулевой.

In [15]:
import numpy as np

def find_max_after_zero(arr):
    """Находит максимальный элемент после нуля"""
    try:
        arr = np.asarray(arr)

        if arr.ndim != 1:
            raise ValueError("Требуется одномерный массив")

        if arr.size == 0:
            raise ValueError("Массив пуст")

        zero_positions = np.where(arr == 0)[0]
        if zero_positions.size == 0:
            return "Нет нулей в массиве"

        valid_positions = zero_positions[zero_positions < len(arr)-1]
        if valid_positions.size == 0:
            return "Нет элементов после нулей"

        after_zero = arr[valid_positions + 1]
        return f"Максимальный элемент после нуля: {np.max(after_zero)}"

    except ValueError as e:
        if "could not convert" in str(e):
            raise ValueError("Массив должен содержать только числа")
        raise
    except Exception as e:
        raise ValueError(f"Ошибка обработки: {e}")

# Правильное тестирование
try:
    # Правильный синтаксис:
    test_arr = np.array([5, 6, 7])  # Запятые обязательны!
    print("Тестируем корректный массив:", test_arr)
    print(find_max_after_zero(test_arr))

except ValueError as e:
    print("Ошибка:", e)

Тестируем корректный массив: [5 6 7]
Нет нулей в массиве


# Задание 6.2.5
2. Загрузите таблицу titanic.csv и найдите ответы на следующие вопросы. Если ответ нужно приводить в процентах, то это должно быть число в интервале от 0 до 100, округленное до двух знаков, знак процента не нужен.

5) Добавьте столбцы FirstNameList с именем пассажира. Обратите внимание, что встречается разный формат хранения имен, в том числе существуют отличия для мужчин и женщин. Имя может быть составное (из нескольких имен) - рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Какое самое популярное женское имя на корабле? Какое самое популярное мужское имя на корабле? У замужних женщин имя берём в скобках

In [ ]:
df_titanic = pd.read_csv('/content/drive/MyDrive/Программирование на языке Python/Lab_Python_6_PIM/titanic.csv')
df_titanic.head()
Survived = (df_titanic["Survived"]==1).sum()/(df_titanic["Survived"]).count()*100
print("Доля выживших пассажиров составила: ", round(Survived,2))

count_men = (df_titanic["Sex"] == "male").sum()
count_women = (df_titanic["Sex"] == "female").sum()
print("Количество мужчин на корабле всего: ", count_men)
print("Количество мужчин на корабле всего: ", count_women)
count_survived_men = ((df_titanic["Survived"]==1)&(df_titanic["Sex"] == "male")).sum()/count_men*100
count_survived_women = ((df_titanic["Survived"]==1)&(df_titanic["Sex"] == "female")).sum()/count_women*100
print("Доля выживших мужчин на корабле: ",round(count_survived_women,2))
print("Доля выживших женщин на корабле: ",round(count_survived_men,2))

a = df_titanic["Pclass"].unique()
total_people = df_titanic["Pclass"].count()
print(sorted(a))
for i in sorted(a):
    print(f"Доля пассажиров {i} класса составила: ", round((df_titanic["Pclass"] == i).sum()/total_people*100,2))

for i in sorted(a):
    print(f"Доля выживших пассажиров {i} класса составила: ", round(((df_titanic["Survived"]==1)&(df_titanic["Pclass"] == i)).sum()/(df_titanic["Pclass"] == i).sum()*100,2))

mean_age = df_titanic["Age"].mean()
median_age = df_titanic["Age"].median()

mean_age_survied = df_titanic[df_titanic["Survived"] == 1]["Age"].mean().round()
median_age_survied  = df_titanic[df_titanic["Survived"] == 1]["Age"].median()

mean_age_unsurvied = df_titanic[df_titanic["Survived"] == 0]["Age"].mean().round()
median_age_unsurvied  = df_titanic[df_titanic["Survived"] == 0]["Age"].median()
print("Средний возраст пассажиров: ", round(mean_age,2))
print("Медианное значение возраста пассажиров: ", round(median_age,2))
print("Средний возраст выживших пассажиров: ", mean_age_survied)
print("Медианное значение возраста выживших пассажиров: ", median_age_survied)
print("Средний возраст невыживших пассажиров: ", mean_age_unsurvied)
print("Медианное значение возраста невыживших пассажиров: ", median_age_unsurvied)


import pandas as pd
import re
df_titanic["Appeal"] = df_titanic["Name"].str.extract(r",\s*([A-Za-z.]+\s*[A-Za-z]+)\.")
df_titanic

# Разделение имени
split_names = df_titanic["Name"].str.split(",", n=1, expand=True)

# Новая функция извлечения женских имен из скобок
def extract_female_name(name_str):
    # Ищем содержимое скобок
    name_in_brackets = re.search(r"\((.*?)\)", name_str)
    if not name_in_brackets:
        return None

    full_name = name_in_brackets.group(1).strip()
    words = full_name.split()

    if not words:
        return None
    # Берем все слова кроме последнего (если слов больше 1)
    return ' '.join(words[:-1]) if len(words) > 1 else words[0]

# Для женщин: применяем новую функцию к Mrs и другим обращениям
df_titanic.loc[df_titanic["Sex"] == "female", "FirstNameList"] = split_names.loc[df_titanic["Sex"] == "female", 1].apply(
    lambda x: extract_female_name(x) if pd.notnull(x) and "(" in x else
             x.split(".", 1)[1].strip() if pd.notnull(x) else None)

# Для мужчин: оставляем прежнюю логику
df_titanic.loc[df_titanic["Sex"] == "male", "FirstNameList"] = split_names.loc[df_titanic["Sex"] == "male", 1].str.split(".", n=1, expand=True)[1].str.strip()

# Очистка имен
df_titanic["FirstNameList"] = df_titanic["FirstNameList"].str.replace(r'[^A-Za-z\s]', '', regex=True).str.strip()
df_titanic["FirstNameList"] = df_titanic["FirstNameList"].apply(
    lambda x: ' '.join([word for word in x.split() if len(word) > 2]) if pd.notnull(x) else x)

# Проверка результата
df_titanic[["Name", "Sex", "Appeal", "FirstNameList"]].head(20)

# Разделение на женские и мужские данные
female_names = df_titanic[df_titanic["Sex"] == "female"]["FirstNameList"]
male_names = df_titanic[df_titanic["Sex"] == "male"]["FirstNameList"]

# Функция для подсчета частот имен
def count_names(names):
    all_names = names.dropna().str.split(expand=True).stack()
    name_counts = all_names.value_counts()
    return name_counts

# Подсчет частот отдельных имен
female_name_counts = count_names(female_names)
male_name_counts = count_names(male_names)

# Функция для подсчета частот составных имен
def count_full_names(names):
    full_name_counts = names.dropna().value_counts()
    return full_name_counts

# Подсчет частот составных имен
female_full_name_counts = count_full_names(female_names)
male_full_name_counts = count_full_names(male_names)

# Получение самых популярных имен с количеством
most_popular_female = (female_name_counts.idxmax(), female_name_counts.max())
most_popular_male = (male_name_counts.idxmax(), male_name_counts.max())
most_popular_female_full = (female_full_name_counts.idxmax(), female_full_name_counts.max())
most_popular_male_full = (male_full_name_counts.idxmax(), male_full_name_counts.max())

# Вывод результатов с количеством
print("САМЫЕ ПОПУЛЯРНЫЕ ИМЕНА:")
print(f"Женское имя (отдельные): {most_popular_female[0]} ({most_popular_female[1]} пассажиров)")
print(f"Женское составное имя: {most_popular_female_full[0]} ({most_popular_female_full[1]} пассажиров)")
print(f"Мужское имя (отдельные): {most_popular_male[0]} ({most_popular_male[1]} пассажиров)")
print(f"Мужское составное имя: {most_popular_male_full[0]} ({most_popular_male_full[1]} пассажиров)")

# Лабораторная работа №9

## Библиотека Scipy. Потоки

**2.** Написать программу, которая вычисляет факториал числа 100 000 и показывает, сколько времени ушло на решение этой задачи. Сначала требуется решить задачу с использованием одного потока, затем с двумя, четырьмя, восьмью. При использовании двух потоков один поток перемножает числа от 1 до 50 000, а второй – от 50 001 до 100 000, а затем результаты перемножаются (можно поэкспериментировать с разделением работы на две части, не обязательно делить интервал пополам, деление осуществлять автоматически). Сравните результаты вычисления факториала (нужно, чтобы результаты совпадали) и время вычисления при каждом способе.

In [ ]:
import math
import time
import threading
import queue

# Функция для вычисления факториала на заданном интервале и записи результата в очередь
def factorial_partial(start, end, result_queue):
    result = 1
    for i in range(start, end + 1):
        result *= i
    result_queue.put(result)

# Функция для вычисления факториала с использованием нескольких потоков
def factorial_multithreaded(n, num_threads):
    # Определяем разделение интервала
    step = n // num_threads
    intervals = [(i * step + 1, (i + 1) * step) for i in range(num_threads)]
    intervals[-1] = (intervals[-1][0], n)  # Убедимся, что последний интервал доходит до n

    # Создаем очередь для получения результатов
    result_queue = queue.Queue()
    threads = []

    # Запускаем потоки
    for start, end in intervals:
        thread = threading.Thread(target=factorial_partial, args=(start, end, result_queue))
        threads.append(thread)
        thread.start()

    # Ожидаем завершения всех потоков
    for thread in threads:
        thread.join()

    # Собираем результаты из очереди
    partial_results = []
    while not result_queue.empty():
        partial_results.append(result_queue.get())

    # Перемножаем результаты из всех потоков
    result = 1
    for partial in partial_results:
        result *= partial
    return result

# Основная функция для сравнения времени выполнения
def main():
    n = 100_000  # Факториал числа 100,000

    # Однопоточное вычисление
    start_time = time.time()
    factorial_single = 1
    for i in range(1, n + 1):
        factorial_single *= i
    single_thread_time = time.time() - start_time
    print(f"Однопоточное вычисление: {single_thread_time:.2f} секунд")

    # Двухпоточное вычисление
    start_time = time.time()
    factorial_two_threads = factorial_multithreaded(n, 2)
    two_thread_time = time.time() - start_time
    print(f"Двухпоточное вычисление: {two_thread_time:.2f} секунд")

    # Четырёхпоточное вычисление
    start_time = time.time()
    factorial_four_threads = factorial_multithreaded(n, 4)
    four_thread_time = time.time() - start_time
    print(f"Четырёхпоточное вычисление: {four_thread_time:.2f} секунд")

    # Восьмипоточное вычисление
    start_time = time.time()
    factorial_eight_threads = factorial_multithreaded(n, 8)
    eight_thread_time = time.time() - start_time
    print(f"Восьмипоточное вычисление: {eight_thread_time:.2f} секунд")

    # Проверка совпадения результатов
    assert factorial_single == factorial_two_threads, "Результаты для двух потоков не совпадают!"
    assert factorial_single == factorial_four_threads, "Результаты для четырёх потоков не совпадают!"
    assert factorial_single == factorial_eight_threads, "Результаты для восьми потоков не совпадают!"
    print("Результаты совпадают для всех потоков!")

if __name__ == "__main__":
    main()

Однопоточное вычисление: 3.68 секунд
Двухпоточное вычисление: 2.06 секунд
Четырёхпоточное вычисление: 0.86 секунд
Восьмипоточное вычисление: 0.56 секунд
Результаты совпадают для всех потоков!


**3.** Написать многопоточную программу, в которой каждый из двух потоков записывает в файл свой номер (1 или 2) 100 000 раз. Файл для обоих потоков один и тот же. Не допускается, чтобы сначала запись выполнял один поток полностью, а потом второй. Автоматически проверить, реально ли в результате в файле записано 100 000 единиц и 100 000 двоек.

In [ ]:
import threading
import traceback

# Количество записей от каждого потока
NUM_WRITES = 100_000
OUTPUT_FILE = "output.txt"

# Счетчики для проверки
ones_count = 0
twos_count = 0

# Блокировка для синхронизации потоков
lock = threading.Lock()

# Функция записи в файл
def write_to_file(thread_id):
    try:
        with open(OUTPUT_FILE, "a") as f:
            for _ in range(NUM_WRITES):
                with lock:  # Синхронизируем доступ к файлу
                    f.write(str(thread_id) + "\n")
    except Exception as e:
        print(f"[Ошибка] Поток {thread_id}: {e}")
        traceback.print_exc()

# Основная функция
def main():
    global ones_count, twos_count

    try:
        # Очищаем файл перед началом работы
        try:
            with open(OUTPUT_FILE, "w") as f:
                pass
        except Exception as e:
            print(f"[Ошибка] Не удалось очистить файл: {e}")
            return

        # Создаем два потока
        thread1 = threading.Thread(target=write_to_file, args=(1,))
        thread2 = threading.Thread(target=write_to_file, args=(2,))

        # Запускаем потоки
        thread1.start()
        thread2.start()

        # Дожидаемся завершения потоков
        thread1.join()
        thread2.join()

        # Проверяем содержимое файла
        try:
            with open(OUTPUT_FILE, "r") as f:
                contents = f.readlines()
        except Exception as e:
            print(f"[Ошибка] Не удалось прочитать файл: {e}")
            return

        # Подсчитываем количество единиц и двоек
        ones_count = contents.count("1\n")
        twos_count = contents.count("2\n")

        print(f"Количество единиц (1): {ones_count}")
        print(f"Количество двоек (2): {twos_count}")

        # Проверяем корректность результата
        if ones_count == NUM_WRITES and twos_count == NUM_WRITES:
            print("Результат корректен: 100,000 единиц и 100,000 двоек.")
        else:
            print("Ошибка: Результат некорректен!")

    except Exception as e:
        print(f"[Ошибка] В процессе выполнения: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    main()

Количество единиц (1): 100000
Количество двоек (2): 100000
Результат корректен: 100,000 единиц и 100,000 двоек.


In [ ]:
import threading
import traceback

# Количество записей от каждого потока
NUM_WRITES = 100_000
OUTPUT_FILE = "output.txt"

# Счетчики для проверки
ones_count = 0
twos_count = 0


# Функция записи в файл
def write_to_file(thread_id):
    try:
        with open(OUTPUT_FILE, "a") as f:
            for _ in range(NUM_WRITES):
                f.write(str(thread_id) + "\n")
    except Exception as e:
        print(f"[Ошибка] Поток {thread_id}: {e}")
        traceback.print_exc()

# Основная функция
def main():
    global ones_count, twos_count

    try:
        # Очищаем файл перед началом работы
        try:
            with open(OUTPUT_FILE, "w") as f:
                pass
        except Exception as e:
            print(f"[Ошибка] Не удалось очистить файл: {e}")
            return

        # Создаем два потока
        thread1 = threading.Thread(target=write_to_file, args=(1,))
        thread2 = threading.Thread(target=write_to_file, args=(2,))

        # Запускаем потоки
        thread1.start()
        thread2.start()

        # Дожидаемся завершения потоков
        thread1.join()
        thread2.join()

        # Проверяем содержимое файла
        try:
            with open(OUTPUT_FILE, "r") as f:
                contents = f.readlines()
        except Exception as e:
            print(f"[Ошибка] Не удалось прочитать файл: {e}")
            return

        # Подсчитываем количество единиц и двоек
        ones_count = contents.count("1\n")
        twos_count = contents.count("2\n")

        print(f"Количество единиц (1): {ones_count}")
        print(f"Количество двоек (2): {twos_count}")

        # Проверяем корректность результата
        if ones_count == NUM_WRITES and twos_count == NUM_WRITES:
            print("Результат корректен: 100,000 единиц и 100,000 двоек.")
        else:
            print("Ошибка: Результат некорректен!")

    except Exception as e:
        print(f"[Ошибка] В процессе выполнения: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    main()

Количество единиц (1): 100000
Количество двоек (2): 100000
Результат корректен: 100,000 единиц и 100,000 двоек.
